# Исследование надежности заемщиков

### Шаг 1. Обзор данных

#### Описание проекта

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

В конце исследования ответим на вопросы:
* Есть ли зависимость между количеством детей и возвратом кредита в срок?
* Есть ли зависимость между семейным положением и возвратом кредита в срок?
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
* Как разные цели кредита влияют на его возврат в срок?

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

#### Описание данных:

* **`children`** — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* **`family_status`** — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* **`debt`** — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

In [1]:
import pandas as pd

In [2]:
bank_statistics = pd.read_csv('/datasets/data.csv')   
# bank_statistics - статистика о платёжеспособности клиентов банка

In [3]:
bank_statistics.info()
#изучаем данные

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
bank_statistics.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


После ознакомления с данными, сразу видим аномалии в столбце `days_employed` - отрицательные значения, и вероятное наличие дубликатов в столбце `education` из-за различного регистра. Далее проверим остальные столбцы на аномалии и дубликаты, которые не так заметны, и исправим проблемы в данных.

### Шаг 2.1 Заполнение пропусков


После ознакомления с данными наблюдаем пропущенные значения в двух столбцах: `days_employed`, `total_income`. 

Возможная причина отсутствия информации в столбцах со стажем и доходом - заемщики не являются зарплатными клиентами банка, поэтому эти данные в статистике банка отсутствуют.

Находим точное количество пропусков:

In [5]:
bank_statistics.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Значения пропущены в столбцах с количественными переменными, мы можем заполнить пропуски медианным значением для каждого из двух столбцов. Такой способ даст более корректный результат в случае присутствия в столбцах аномальных значений, чем заполнение пропусков средним значением по столбцу.

In [6]:
days_employed_median = bank_statistics['days_employed'].median()
total_income_median = bank_statistics['total_income'].median()

In [7]:
# заполняем отсутствующие значения
bank_statistics['days_employed'] = bank_statistics['days_employed'].fillna(days_employed_median)
bank_statistics['total_income'] = bank_statistics['total_income'].fillna(total_income_median)

In [8]:
# проверка
bank_statistics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [9]:
bank_statistics.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Шаг 2.2 Проверка данных на аномалии и исправления.

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. 
Например, отрицательное количество дней трудового стажа в столбце `days_employed`. 


Возможно, причина таких аномалий в ошибках при внесении информации в клиентскую базу.

In [10]:
bank_statistics.groupby('days_employed').sum().head()

,children,dob_years,education_id,family_status_id,debt,total_income
days_employed,,,,,,
-18388.949901,1,61,1,0,0,186178.934089
-17615.563266,0,61,1,0,0,122560.741753
-16593.472817,0,60,0,0,0,124697.846781
-16264.699501,0,59,1,0,0,51238.967133
-16119.687737,0,64,1,0,0,91527.685995


Воспользуемся методом `apply` и функцией `abs`, чтобы избавиться от отрицательных значений в столбце `days_employed`.

In [11]:
bank_statistics['days_employed'] = bank_statistics['days_employed'].apply(abs)

In [12]:
# проверка
bank_statistics.groupby('days_employed').sum().head()

,children,dob_years,education_id,family_status_id,debt,total_income
days_employed,,,,,,
24.141633,1,31,1,0,1,166952.415427
24.240695,0,32,0,4,0,124115.373655
30.195337,2,47,1,1,0,231461.185606
33.520665,0,43,1,4,1,128555.897209
34.701045,1,31,0,0,0,90557.994311


Возможные дальнейшие преобразования в столбце `days_employed` не проводим, т.к. эти данные не пригодятся нам в дальнейшем исследовании.

В столбце `children` найден артефакт - отрицательное кол-во детей. Возможно, это ошибка ввода данных. 
Также среди значений есть подозрительно большое число 20, возможно, что это ошибочно записанное число 2.

Исправим аномалии с помощью метода `replace`, заменим значение `-1` на `1` и значение `20` на `2`.

In [13]:
bank_statistics['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [14]:
bank_statistics['children'] = bank_statistics['children'].replace(-1, 1)

In [15]:
bank_statistics['children'] = bank_statistics['children'].replace(20, 2)

In [16]:
# проверка
bank_statistics['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Находим аномалию в столбце `gender`. Т.к. сама строка имеет ценные для исследования данные, то удалять ее не рационально. 

In [17]:
bank_statistics['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

Заменим значение в строке на `'unknown'` методом `replace`.

In [18]:
bank_statistics[bank_statistics['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


In [19]:
bank_statistics['gender'] = bank_statistics['gender'].replace('XNA', 'unknown')

In [20]:
# проверка
bank_statistics['gender'].unique()

array(['F', 'M', 'unknown'], dtype=object)

In [21]:
# аномалий в данных не обнаружено
bank_statistics['debt'].unique()

array([0, 1])

Находим ошибку в столбце `dob_years` - на некоторых строках возраст указан как 0 лет.

In [22]:
bank_statistics.groupby(['dob_years', 'income_type']).sum()

children  days_employed  education_id  \
dob_years income_type                                          
0         госслужащий         5   3.527257e+04             2   
          компаньон          12   3.460632e+04             9   
          пенсионер           0   6.166748e+06            14   
          сотрудник          33   1.013725e+05            43   
19        госслужащий         0   5.099699e+02             2   
...                         ...            ...           ...   
73        пенсионер           1   2.194711e+06             9   
74        компаньон           0   1.729633e+03             1   
          пенсионер           0   1.439163e+06             3   
          сотрудник           0   6.682868e+03             1   
75        госслужащий         0   1.678970e+03             1   

                       family_status_id  debt  total_income  
dob_years income_type                                        
0         госслужащий                 8     0  8.818397e+05  
          компаньон                  18     1  3.632562e+06  
          пенсионер                  33     1  2.789434e+06  
          сотрудник                  66     6  8.555097e+06  
19        госслужащий                 4     0  7.578742e+04  
...                                 ...   ...           ...  
73        пенсионер                   7     0  7.665975e+05  
74        компаньон                   0     0  1.248301e+05  
          пенсионер                   2     0  2.777069e+05  
          сотрудник                   1     0  9.894591e+04  
75        госслужащий                 2     0  1.532826e+05  

[221 rows x 6 columns]

In [23]:
# находим медианное значение для категории 'пенсионер'
bank_statistics[(bank_statistics['income_type'] == 'пенсионер') & (bank_statistics['dob_years'] != 0)]['dob_years'].median()

60.0

In [24]:
# находим медианное значение для остальных категорий
bank_statistics[(bank_statistics['income_type'] != 'пенсионер') & (bank_statistics['dob_years'] != 0)]['dob_years'].median()

39.0

Применим два способа замены: через методы `loc` и `replace`, т.к. одного метода `replace` недостаточно.

In [25]:
bank_statistics.loc[((bank_statistics['dob_years'] == 0) & (bank_statistics['income_type'] == 'пенсионер')), 'dob_years'] = 60

In [26]:
bank_statistics['dob_years'] = bank_statistics['dob_years'].replace(0, 39)

In [27]:
# проверка
bank_statistics.groupby(['dob_years', 'income_type']).sum()

children  days_employed  education_id  \
dob_years income_type                                          
19        госслужащий         0   5.099699e+02             2   
          компаньон           0   3.675963e+03             9   
          сотрудник           0   5.255252e+03             8   
20        госслужащий         0   1.291342e+03             1   
          компаньон           2   8.879091e+03            17   
...                         ...            ...           ...   
73        пенсионер           1   2.194711e+06             9   
74        компаньон           0   1.729633e+03             1   
          пенсионер           0   1.439163e+06             3   
          сотрудник           0   6.682868e+03             1   
75        госслужащий         0   1.678970e+03             1   

                       family_status_id  debt  total_income  
dob_years income_type                                        
19        госслужащий                 4     0  7.578742e+04  
          компаньон                  15     0  8.398840e+05  
          сотрудник                  13     1  6.101043e+05  
20        госслужащий                 8     0  3.198307e+05  
          компаньон                  39     3  2.085864e+06  
...                                 ...   ...           ...  
73        пенсионер                   7     0  7.665975e+05  
74        компаньон                   0     0  1.248301e+05  
          пенсионер                   2     0  2.777069e+05  
          сотрудник                   1     0  9.894591e+04  
75        госслужащий                 2     0  1.532826e+05  

[217 rows x 6 columns]

### Шаг 2.3. Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype`. 

Дабавим конструкцию `try...except` на случай возможной ошибки.

In [28]:
bank_statistics['total_income'].dtypes

dtype('float64')

In [29]:
try:
    bank_statistics['total_income'] = bank_statistics['total_income'].astype('int')
except: 
    print('error')

In [30]:
# проверка
bank_statistics['total_income'].dtypes

dtype('int64')

### Шаг 2.4. Удаление дубликатов

С помощью метода `unique` будем находить уникальные значения в столбцах для проверки наличия неявных дубликатов в них.

In [31]:
bank_statistics['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

С названиями в столбце `'education'` всё в порядке, но небходимо исправить регистр написания.

Используем метод `lower`.

In [32]:
bank_statistics['education'] = bank_statistics['education'].str.lower()

In [33]:
# проверка
bank_statistics['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [34]:
# неявных дубликатов не обнаружено
bank_statistics['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [35]:
# неявных дубликатов не обнаружено
bank_statistics['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

Когда неявные дубликаты устранены, найдем полные дубликаты и удалим их методом `drop_duplicates` и `reset_index`, чтобы исправить индексацию.

In [36]:
bank_statistics.duplicated().sum()

71

In [37]:
bank_statistics = bank_statistics.drop_duplicates().reset_index(drop=True)

In [38]:
# проверка
bank_statistics.duplicated().sum()

0

Причин возникновения дубликатов может быть несколько, предположительно, в банке может отсутствовать регламент записи данных клиентов для сотрудников, или данные собирают из информации, что клиент сам о себе написал. В обоих случаях возможно возникновение большого кол-ва неявных дубликатов. Также дубликаты могли появиться в следствии сбоев в системе банка.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма

Создадим два новых датафрейма со столбцами:

1) `education_id` и `education`

2) `family_status_id` и `family_status`

In [39]:
education = bank_statistics[['education_id', 'education']]

In [40]:
education.duplicated().sum()

21449

In [41]:
education = education.drop_duplicates().reset_index(drop=True)  #удалим дубликаты

In [42]:
education.duplicated().sum()

0

In [43]:
# проверка
education

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [44]:
family_status = bank_statistics[['family_status_id', 'family_status']]

In [45]:
family_status.duplicated().sum()

21449

In [46]:
family_status = family_status.drop_duplicates().reset_index(drop=True)  #удалим дубликаты

In [47]:
family_status.duplicated().sum()

0

In [48]:
# проверка
family_status

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


Удалим из исходного датафрейма столбцы `education` и `family_status`, оставив только их идентификаторы: `education_id` и `family_status_id`.

In [49]:
bank_statistics = bank_statistics.drop(columns=['education', 'family_status'])

In [50]:
# проверка
bank_statistics.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.6. Категоризация дохода

На основании диапазонов, указанных ниже, создадим столбец `total_income_category` с категориями:
* 0–30000 — 'E'
* 30001–50000 — 'D'
* 50001–200000 — 'C'
* 200001–1000000 — 'B'
* 1000001 и выше — 'A'



In [51]:
def total_income_category(total_income): # функция для категоризации значений total_income
       
    if total_income <= 30_000:
        return 'E'
    elif total_income <= 50_000:
        return 'D'
    elif total_income <= 200_000:
        return 'C'
    elif total_income <= 1_000_000:
        return 'B'
    elif total_income > 1_000_000:
        return 'A'
    return 'other'

In [52]:
bank_statistics['total_income_category'] = bank_statistics['total_income'].apply(total_income_category)

In [53]:
# проверка
bank_statistics.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


In [54]:
# проверка
bank_statistics['total_income_category'].sort_values().unique()

array(['A', 'B', 'C', 'D', 'E'], dtype=object)

### Шаг 2.7. Категоризация целей кредита

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории: операции с автомобилем, операции с недвижимостью, проведение свадьбы, получение образования.

In [55]:
def purpose_rename(purpose):  
       
    if 'авто' in purpose:
        return 'операции с автомобилем' 
    elif 'недвиж' in purpose or 'жиль' in purpose:
        return 'операции с недвижимостью' 
    elif 'образов' in purpose:
        return 'получение образования'
    elif 'свадьб' in purpose:
        return 'проведение свадьб' 
    return 'error' 

In [56]:
bank_statistics['purpose_category'] = bank_statistics['purpose'].apply(purpose_rename)

In [57]:
# проверка
bank_statistics.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьб


In [58]:
# проверка
bank_statistics['purpose_category'].unique()

array(['операции с недвижимостью', 'операции с автомобилем',
       'получение образования', 'проведение свадьб'], dtype=object)

### Ответы на вопросы.

Для анализа данных используем метод `pivot_table`, т.к. он позволяет создавать более удобные для анализа таблицы.

Также посчитаем столбец `'ratio'` с отношением кол-ва задолженностей ко всем значениям в группе для более точного сравнения категорий.

##### Вопрос 1:

**Есть ли зависимость между количеством детей и возвратом кредита в срок?**

In [59]:
children_ratio = bank_statistics.pivot_table(index='children', values='debt', aggfunc=('sum', 'count', 'mean'))

In [60]:
children_ratio['ratio'] = children_ratio['mean']/children_ratio['mean'].sum()
children_ratio = children_ratio.style.format({'ratio': '{:.1%}'})

In [61]:
children_ratio

,count,mean,sum,ratio
children,,,,
0,14091,0.0754382,1063,17.1%
1,4855,0.0916581,445,20.8%
2,2128,0.0949248,202,21.5%
3,330,0.0818182,27,18.5%
4,41,0.097561,4,22.1%
5,9,0,0,0.0%


##### Вывод 1:

Зависимость между количеством детей и возвратом кредита в срок:
- Заёмщики без детей оказались немного надежнее для банка, чем клиенты с детьми. 
- У клиентов с 5-ю детьми задолженностей не обнаружено, что скорее всего связано с недостаточным для анализа количеством клиентов в этой категории, их всего 9.

##### Вопрос 2:

**Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [62]:
family_status_ratio = bank_statistics.pivot_table(index=['family_status_id'], values='debt', aggfunc=('sum', 'count', 'mean'))

In [63]:
family_status_ratio['ratio'] = family_status_ratio['mean']/family_status_ratio['mean'].sum()
family_status_ratio = family_status_ratio.style.format({'ratio': '{:.1%}'})

In [64]:
family_status

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [65]:
family_status_ratio

,count,mean,sum,ratio
family_status_id,,,,
0,12339,0.0754518,931,18.7%
1,4151,0.0934715,388,23.2%
2,959,0.0656934,63,16.3%
3,1195,0.0711297,85,17.6%
4,2810,0.0975089,274,24.2%


##### Вывод 2:

Зависимость между семейным положением и возвратом кредита в срок: 

- Клиенты, которые не состоят в браке, чаще имели задолженность по выплатам, в отличие от клиентов, которые находятся/находились в браке, те реже имели задолженность.

##### Вопрос 3: 

**Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [66]:
total_income_ratio = bank_statistics.pivot_table(index='total_income_category', values='debt', aggfunc=('sum', 'count', 'mean'))

In [67]:
total_income_ratio['ratio'] = total_income_ratio['mean']/total_income_ratio['mean'].sum()
total_income_ratio = total_income_ratio.style.format({'ratio': '{:.1%}'})

In [68]:
total_income_ratio

,count,mean,sum,ratio
total_income_category,,,,
A,25,0.08,2,20.7%
B,5041,0.0706209,356,18.3%
C,16016,0.0849151,1360,22.0%
D,350,0.06,21,15.5%
E,22,0.0909091,2,23.5%


##### Вывод 3:

Зависимость между уровнем дохода и возвратом кредита в срок:

- Чаще кредит в срок возвращали клиенты с ежемесячным доходом 30к-50к ('D') и 200к-1000к ('B').
- Клиентов с доходом менее 30.000 в месяц ('Е') немного, возможно, клиенты из этой категории редко берут или получают одобрение на кредит. Но те, кто брали, реже возвращали долг вовремя.
- Также мало данных по клиентам с доходом свыше 1 млн. ('A').

##### Вопрос 4:

**Как разные цели кредита влияют на его возврат в срок?**

In [69]:
purpose_ratio = bank_statistics.pivot_table(index='purpose_category', values='debt', aggfunc=('sum', 'count', 'mean'))

In [70]:
purpose_ratio['ratio'] = purpose_ratio['mean']/purpose_ratio['mean'].sum()
purpose_ratio = purpose_ratio.style.format({'ratio': '{:.1%}'})

In [71]:
purpose_ratio

,count,mean,sum,ratio
purpose_category,,,,
операции с автомобилем,4306,0.0935903,403,27.7%
операции с недвижимостью,10811,0.0723337,782,21.4%
получение образования,4013,0.0922003,370,27.3%
проведение свадьб,2324,0.0800344,186,23.7%


##### Вывод 4:

Зависимость между целью кредита и возвратом кредита в срок:

- Клиенты с целями, связанными с получением образования и операциями с автомобилем, реже возвращают кредит в срок.
- Кредиты на операции с недвижимостью выплачивались более своевременно.

## Общий вывод:

Исходя из полученных данных, можем собрать образ исправного плательщика, а также клиента, который с большей вероятностью станет задолженником по кредиту.

- С меньшей вероятностью долг по кредиту образуется у клиента без детей, который состоит в браке, имеет ежемесячный доход в диапазоне 200к-1000к рублей и берет кредит на операции с недвижимостью.


- С большей вероятностью долг по кредиту образуется у клиента с детьми, не состоящего в браке, с ежемесячным доходом до 30к и с целью кредита, связанной с образованием или операциями с автомобилем.

